In [1]:
import tensorflow as tf

In [23]:
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


x不是一个特定的值，而是一个占位符placeholder。在tensorflow运行计算时输入这个值。我们希望能够输入任意数量的MNIST图像，每一张图展平成784维的向量。我们用2维的浮点数张量来表示这些图，[None,784]。

In [28]:
x=tf.placeholder(tf.float32,[None,784])
y_=tf.placeholder(tf.float32,[None,10])

In [31]:
W=tf.Variable(tf.zeros([784,10]))
b=tf.Variable(tf.zeros([10]))

sess=tf.Session()
sess.run(tf.global_variables_initializer())
##实现模型
y = tf.nn.softmax(tf.matmul(x,W)+b)
cross_entropy=-tf.reduce_mean(tf.multiply(y_,tf.log(y)))

#### 训练模型  
我们已经定义好模型和训练用的损失函数。用最速下降法让交叉熵下降，步长为0.01.

In [34]:
train_step=tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [61]:
for i in range(1000):
    batch = mnist.train.next_batch(50)
    sess.run(train_step,feed_dict={x:batch[0], y_:batch[1]})

In [62]:
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,'float'))  

In [63]:
print(sess.run(accuracy,feed_dict={x:mnist.test.images,y_:mnist.test.labels}))

0.8426


### 建立一个多层卷积网络  
在MNIST上只有84%的正确率，比较糟糕。我们用一个稍微复杂的模型：卷积神经网络来改善效果。  
#### 权重初始化  
为了创建这个模型，我们需要创建大量的权重和偏置项。这个模型中的权重在初始化时应该加入少量的噪声来打破对称性和提督消失。由于我们使用的是relu神经元，因此比较好的做法是用一个较小的正数来初始化偏置项，以避免神经元节点输出恒为0的问题。

In [50]:
def weight_variable(shape):
    initial=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial=tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

#### 卷积和池化
我们的卷积使用1步长，0边距的模版，保证输出和输入是同一个大小。我们的池化用简单传统的2X2大小的模版做max pooling。为了代码更简洁，我们把这部分抽象成一个函数。

In [65]:
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding="SAME")

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')


### 第一层卷积
它由一个卷积接一个max pooling完成。卷积在每个5x5的patch中算出32个特征。卷积的权重张量形状是[5,5,1,32]，前两个维度是patch的大小，接着是输入的通道数目，最后是输出的通道数目。而对应于每一个输出通道都有一个对应的偏置量。

In [66]:
W_conv1=weight_variable([5,5,1,32])
b_conv1=bias_variable([32])

为了用这一层，我们把x变成一个4d向量，其第2，3维对应图片的宽和高，最后一维代表图片的颜色通道(因为是灰度图所以这里的通道数为1，如果是rgb彩色图，则为3)

In [67]:
x_image=tf.reshape(x,[-1,28,28,1])

我们把x_image和权值向量进行卷积，加上偏置项，然后应用relu激活函数，最后进行maxpooling.

In [69]:
h_conv1=tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
h_pool1=max_pool_2x2(h_conv1)

### 第二层卷积  
为了构建一个更深的网络，我们会把几个类似的层堆叠起来，第二层中，每个5x5的patch会得到64个特征。

In [73]:
W_conv2=weight_variable([5,5,32,64])
b_conv2=bias_variable([64])

h_conv2=tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
h_pool2=max_pool_2x2(h_conv2)

### 密集链接层
现在，图片尺寸减小到7x7,我们加入一个有1024个神经元的全连接层，用于处理整个图片。我们把池化层输出的张量reshape成一些向量，乘上权重矩阵，加上偏置，然后对其使用relu.

In [75]:
W_fc1=weight_variable([7*7*64,1024])
b_fc1=bias_variable([1024])

h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*64])
h_fc1=tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

### Dropout
为了减少过拟合，我们在输出层之前加入dropout.我们用一个placeholder来代表一个神经元的输出在dropout中保持不变的概率。这样我们可以在训练过程中启用dropout,在测试过程中关闭dropout。Tensorflow的tf.nn.dropout操作除了可以屏蔽神经元的输出外，还会自动处理神经元输出值的scale,所以使用dropout时可以不考虑scale。

In [76]:
keep_prob=tf.placeholder('float')
h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob)

### 输出层
最后，我们添加一个softmax层，就像前面的单层softmax regression一样

In [77]:
W_fc2=weight_variable([1024,10])
b_fc2=bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)

### 训练和评估模型
为了进行训练和评估，我们使用与之前简单的单层softmax神经网络模型几乎相同的一套代码，只是我们会用更复杂的ADAM优化器来做梯度最速下降，在feed_dict中加入额外的参数keep_prob来控制dropout比例。然后每100次迭代输出一次日志。

In [82]:
cross_entropy=-tf.reduce_sum(tf.multiply(y_,tf.log(y_conv)))
train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction=tf.equal/(tf.argmax(y_conv,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,'float'))

sess.run(tf.global_variables_initializer())

for i in range(20000):
    batch=mnist.train.next_batch(50)
    if i %100==0:
        train_accuracy = sess.run(accuracy,feed_dict={
            x:batch[0],y_:batch[1],keep_prob:1.0
        })
        print('step %d, train_accuracy %g'%(i,train_accuracy))
        
    sess.run(train_step,feed_dict={x:batch[0],y_:batch[1],keep_prob:0.5})


TypeError: unsupported operand type(s) for /: 'function' and 'tuple'

In [90]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())


In [ ]:
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy =sess.run(accuracy,feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    sess.run(train_step,feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%(sess.run(accuracy,feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})))

step 0, training accuracy 0.94
step 100, training accuracy 0.9
step 200, training accuracy 0.96
step 300, training accuracy 0.94
step 400, training accuracy 0.94
step 500, training accuracy 0.96
step 600, training accuracy 1
step 700, training accuracy 0.96
step 800, training accuracy 0.92
step 900, training accuracy 1
step 1000, training accuracy 1
step 1100, training accuracy 0.96
step 1200, training accuracy 0.96
step 1300, training accuracy 0.98
step 1400, training accuracy 0.96
step 1500, training accuracy 1
step 1600, training accuracy 0.98
step 1700, training accuracy 1
step 1800, training accuracy 0.96
step 1900, training accuracy 0.98
step 2000, training accuracy 0.98
step 2100, training accuracy 0.96
step 2200, training accuracy 0.98
step 2300, training accuracy 1
step 2400, training accuracy 0.98
step 2500, training accuracy 0.94
step 2600, training accuracy 0.98
step 2700, training accuracy 0.96
step 2800, training accuracy 1
step 2900, training accuracy 0.98
step 3000, tra